### 1. Для реализованной модели градиентного бустинга построить графики зависимости ошибки от количества деревьев в ансамбле и от максимальной глубины деревьев. Сделать выводы о зависимости ошибки от этих параметров.

In [1]:
%matplotlib inline
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_diabetes
import matplotlib.pyplot as plt
import numpy as np

In [2]:
X, y = load_diabetes(return_X_y=True)
X.shape, y.shape

((442, 10), (442,))

In [3]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)

In [4]:
def gb_predict(X, trees_list, eta):
    
#     predictions = np.zeros(X.shape[0])
#     for i, x in enumerate(X):
#         prediction = 0
#         for alg in trees_list:
#             prediction += eta * alg.predict([x])[0]
#         predictions[i] = prediction

    predictions = np.array(
        [sum([eta * alg.predict([x])[0] for alg in trees_list]) for x in X]
    )
    
    return predictions

In [5]:
def mean_squared_error(y_real, prediction):
    return sum((y_real - prediction)**2) / len(y_real)

In [6]:
def residual(y, z):
    return -(z - y)

In [7]:
def gb_fit(n_trees, max_depth, X_trian, X_test, y_train, y_test, eta):
    
    trees = []
    
    train_errors = []
    test_errors = []
    
    for i in range(n_trees):

        tree = DecisionTreeRegressor(max_depth=max_depth, random_state=42)

        if len(trees) == 0:

            tree.fit(X_train, y_train)

            train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, eta)))

        else:

            target = gb_predict(X_trian, trees, eta)

            tree.fit(X_train, residual(y_train, target))

            train_errors.append(mean_squared_error(y_train, gb_predict(X_train, trees, eta)))
            test_errors.append(mean_squared_error(y_test, gb_predict(X_test, trees, eta)))

        trees.append(tree)
    
    return trees, train_errors, test_errors

In [ ]:
max_depth = 10
trees_n = 50
eta = 1
trees, train_errors, test_errors = gb_fit(trees_n, max_depth, X_train, X_test, y_train, y_test, eta)

In [ ]:
def print_diagram(trees_n, train_err, test_err):
    plt.xlabel('Iteration number')
    plt.ylabel('MSE')
    plt.xlim(0, trees_n)
    plt.plot(list(range(trees_n)), train_err, label='train error')
    plt.plot(list(range(trees_n)), test_err, label='test error')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
print_diagram(len(trees), train_errors, test_errors)

In [ ]:
max_depth = 20
trees, train_errors, test_errors = gb_fit(trees_n, max_depth, X_train, X_test, y_train, y_test, eta)
print_diagram(len(trees), train_errors, test_errors)

In [ ]:
max_depth = 5
trees, train_errors, test_errors = gb_fit(trees_n, max_depth, X_train, X_test, y_train, y_test, eta)
print_diagram(len(trees), train_errors, test_errors)

In [ ]:
max_depth = 3
trees, train_errors, test_errors = gb_fit(trees_n, max_depth, X_train, X_test, y_train, y_test, eta)
print_diagram(len(trees), train_errors, test_errors)

In [ ]:
max_depth = 2
trees, train_errors, test_errors = gb_fit(trees_n, max_depth, X_train, X_test, y_train, y_test, eta)
print_diagram(len(trees), train_errors, test_errors)

In [ ]:
max_depth = 1
trees, train_errors, test_errors = gb_fit(trees_n, max_depth, X_train, X_test, y_train, y_test, eta)
print_diagram(len(trees), train_errors, test_errors)

In [ ]:
eta = 0.1
trees, train_errors, test_errors = gb_fit(trees_n, max_depth, X_train, X_test, y_train, y_test, eta)
print_diagram(len(trees), train_errors, test_errors)

### 2. * Модифицировать реализованный алгоритм градиентного бустинга, чтобы получился стохастический градиентный бустинг. Размер подвыборки принять равным 0.5. Сравнить на одном графике кривые изменения ошибки на тестовой выборке в зависимости от числа итераций.